# CP 5 Aprendizaje de Máquinas

---


## Support Vector Machine y Cross Validation


## Ejercicio 1: Preprocesamiento del Dataset

Vamos a trabajar con un dataset que contiene información acerca de pacientes (*gender, marital status, smoking status, age, etc*) con el ojetivo de predecir si es probable que tenga un ataque al corazón (*stroke*).
Además se incorpora de manera artificial la característica *doctor* que representa el doctor que recolectó los datos, la cual más adelante será usada para agrupar los datos.

In [ ]:
import pandas as pd
import numpy as np

def get_prep_data():
    df = pd.read_csv(
        "./resources/healthcare-dataset-stroke-data.csv"
    )
    df["ever_married"] = (
        df["ever_married"].replace("Yes", True).replace("No", False)
    )
    df["gender"] = df["gender"].astype("category")
    df["smoking_status"] = df["smoking_status"].astype("category")
    df["Residence_type"] = df["Residence_type"].astype("category")
    df["work_type"] = df["work_type"].astype("category")
    df["doctor"] = np.random.randint(0, 8, size=len(df))
    
    return df


df = get_prep_data()

A continuación exploremos la estructura del dataset en cuestión:

In [ ]:
df.head()

Se observan variables categóricas las cuales es necesario codificar 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from typing import List

In [ ]:
def encode_features(df: pd.DataFrame, features: List[str]):
    le = LabelEncoder()
    for feat in features:
        df[feat] = le.fit_transform(df[feat])

In [ ]:
encode_features(df, ['gender', 'work_type', 'Residence_type', 'smoking_status'])

Observemos el dataset resultante:

In [ ]:
df.head()

Comprobemos si para todas las muestras observadas en el dataset se tienen valores válidos. Para ello no deben existir valores `NaN`

In [ ]:
df.isna().any()

En caso de existir valores `NaN` estos deben ser eliminados o reemplazados por algún otro valor que sea válido y represente la ausencia de información para esa muestra.

In [ ]:
df['bmi'].fillna(-1, inplace=True)
df.head()

Comprobemos la cantidad de muestras correspondientes a cada clase (1 para stroke, 0 para no stroke)

In [ ]:
df['stroke'].value_counts()

Como se puede apreciar estamos en presencia de un dataset altamente desbalanceado

La función `get_X_y` separa el dataset según las columnas elegidas como características y la columna *stroke* que representa el objetivo a predecir. Además se separa la columna *doctor* por la cual se quiere agrupar.

In [ ]:
def get_X_y(train):
    FEATURES = [
        "gender",
        "age",
        "hypertension",
        "heart_disease",
        "ever_married",
        "work_type",
        "Residence_type",
        "avg_glucose_level",
        "bmi",
        "smoking_status",
    ]

    GROUPS = "doctor"

    TARGET = "stroke"

    X = train[FEATURES]
    y = train[TARGET]
    groups = train[GROUPS]
    return X, y, groups

## Ejercicio 2: Clasificación usando Support Vector Machine

Se divide el dataset en conjunto de entrenamiento y conjunto de prueba.

In [ ]:
from sklearn.model_selection import train_test_split

X, y, groups = get_X_y(df)
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1)

Proponemos usar la clase `SVC` como implementación para Support Vector Machine, la cual podemos encontrar en `sklearn`, para entrenar un clasificador en el conjunto de entrenamiento obtenido. Por defecto `SVC` usa `rbf` como función de kernel.

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf = SVC()
clf.fit(X_tr, y_tr)

### Métricas



In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

### Baseline

In [ ]:
baseline = np.random.randint(0,2,y_val.shape)

acc_score = accuracy_score(y_val, baseline)
auc_score = roc_auc_score(y_val, baseline)
print(
    f"Our baseline on the holdout set is {acc_score:0.4f} and AUC is {auc_score:0.4f}"
)

In [ ]:
pred = clf.predict(X_val)

acc_score_1 = accuracy_score(y_val, pred)
auc_score_1 = roc_auc_score(y_val, pred)
print(
    f"Accuracy on the validation set is {acc_score_1:0.4f} and AUC is {auc_score_1:0.4f}"
)

### Cross Validation

We will learn:
1. Example where not using cross validation causes us to believe our model is better than it is.
2. Show the different cross validation techniuqes and when to apply them.
3. Run our example from part 1 using the correct cross validation technique.

In [ ]:
import matplotlib.pylab as plt
from matplotlib.patches import Patch

plt.style.use('seaborn-white')

cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

def visualize_groups(classes, groups, name):
    # Visualize dataset groups
    fig, ax = plt.subplots()
    ax.scatter(
        range(len(groups)),
        [0.5] * len(groups),
        c=groups,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.scatter(
        range(len(groups)),
        [3.5] * len(groups),
        c=classes,
        marker="_",
        lw=50,
        cmap=cmap_data,
    )
    ax.set(
        ylim=[-1, 5],
        yticks=[0.5, 3.5],
        yticklabels=["Data\ngroup", "Data\nclass"],
        xlabel="Sample index",
    )


def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=25):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data
    )

    # Formatting
    yticklabels = list(range(n_splits)) + ["class", "group"]
    ax.set(
        yticks=np.arange(n_splits + 2) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 2.2, -0.2],
        xlim=[0, X.shape[0]],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax


def plot_cv(cv, X, y, groups, n_splits=5):
    this_cv = cv(n_splits=n_splits)
    fig, ax = plt.subplots(figsize=(15, 5))
    plot_cv_indices(this_cv, X, y, groups, ax, n_splits)

    ax.legend(
        [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
        ["Testing set", "Training set"],
        loc=(1.02, 0.8),
    )
    plt.tight_layout()
    fig.subplots_adjust(right=0.7)
    plt.show()
    
def get_fake_X_y():
    # Fake Generate the class/group data for an example
    n_points = 100
    X_ = np.random.randn(100, 10)

    percentiles_classes = [0.1, 0.9]
    y_ = np.hstack([[ii] * int(100 * perc) for ii, perc in enumerate(percentiles_classes)])

    # Evenly spaced groups repeated once
    groups_ = np.hstack([[ii] * 10 for ii in range(10)])
    return X_, y_, groups_

In [ ]:
from sklearn.model_selection import (
    TimeSeriesSplit,
    KFold,
    StratifiedKFold,
    GroupKFold,
    StratifiedGroupKFold
)

# KFold
- Split dataset into k consecutive folds (without shuffling by default).

In [ ]:
kf = KFold()
X_, y_, groups_ = get_fake_X_y()
plot_cv(KFold, X_, y_, groups_)

# Stratified KFold
- KFold but the folds are made by preserving the percentage of samples for each class.

In [ ]:
skf = StratifiedKFold()
X_, y_, groups_ = get_fake_X_y()
plot_cv(StratifiedKFold, X_, y_, groups_)

# Group KFold

The folds are approximately balanced in the sense that the number of distinct groups is approximately the same in each fold.

In [ ]:
gkf = GroupKFold()
X_, y_, groups_ = get_fake_X_y()
plot_cv(GroupKFold, X_, y_, groups_)

In [ ]:
# plot_cv(GroupKFold, )

# Stratified Group KFold

The difference between GroupKFold and StratifiedGroupKFold is that the former attempts to create balanced folds such that the number of distinct groups is approximately the same in each fold, whereas StratifiedGroupKFold attempts to create folds which preserve the percentage of samples for each class as much as possible given the constraint of non-overlapping groups between splits.

In [ ]:
gskf = StratifiedGroupKFold()
X_, y_, groups_ = get_fake_X_y()
np.random.shuffle(y_)
plot_cv(StratifiedGroupKFold, X_, y_, groups_)

# Time Series Split

In [ ]:
tss = TimeSeriesSplit()
X_, y_, groups_ = get_fake_X_y()
np.random.shuffle(y_)
plot_cv(TimeSeriesSplit, X_, y_, groups_)

In [ ]:
# Grouping the groups together
indeces = list(groups.groupby(groups).groups.values())
flat_index = [item for sublist in indeces for item in sublist]

In [ ]:
X_, y_, groups_ = X.loc[flat_index], y.loc[flat_index], groups.loc[flat_index]

In [ ]:
plot_cv(StratifiedGroupKFold, X_, y_, groups_)

## Ejercicio 3: Cross Validation aplicado a SVM 
1. Small/imblanced -> Stratified
2. Group
3. Shuffle in on

`StratifiedGroupKFold` is a good choice for this situation.

In [ ]:
def train_with_cv(clf):
    sgk = StratifiedKFold(n_splits=5, shuffle=True, random_state=529)

    X, y, groups = get_X_y(df)

    fold = 0
    aucs = []
    for train_idx, val_idx in sgk.split(X, y, groups):
        X_tr = X.loc[train_idx]
        y_tr = y.loc[train_idx]

        X_val = X.loc[val_idx]
        y_val = y.loc[val_idx]

        # Fit Model on Train
        clf.fit(X_tr, y_tr)
        pred = clf.predict(X_val)
        auc_score = roc_auc_score(y_val, pred)
        print(f"======= Fold {fold} ========")
        print(
            f"Our AUC on the validation set is {auc_score:0.4f}"
        )
        fold += 1
        aucs.append(auc_score)
    oof_auc = np.mean(aucs)
    print(f'Our out of fold AUC score is {oof_auc:0.4f}')

In [ ]:
train_with_cv(SVC())

Our averaged out of fold score is a much better estimation of how our model will perform on unseen data.

## Ejercicio 4: Cross Validation para el resto de los clasificadores

In [ ]:
from sklearn.naive_bayes import GaussianNB

train_with_cv(GaussianNB())

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

train_with_cv(KNeighborsClassifier())

In [ ]:
from sklearn.tree import DecisionTreeClassifier

train_with_cv(DecisionTreeClassifier(criterion="entropy"))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

train_with_cv(RandomForestClassifier(criterion="entropy"))

In [ ]:
from sklearn.linear_model import LogisticRegression

train_with_cv(LogisticRegression(solver='liblinear'))